In [1]:
import PyPDF2
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import os
import re
import itertools
#source: https://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import logging
from datetime import datetime
import csv

def convertPdfToTxt_m2(fullPath):
    pdf_file = open(fullPath, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    page = read_pdf.getPage(0)
    page_content = page.extractText()
    return page_content

#read and extract text from one pdf file using pdfminer
def convertPdfToTxt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [2]:
def groupText(content):
    #topicList = ['RESULTS REVIEW:', 'ASSESSMENT:', 'MORE DETAILS:'] 
    topicList = ['RAHSIA', 'ASSESSMENT:', 'MORE DETAILS:']
    contentList = content.split('\n')
    if topicList[0] in content:
        review = ' '.join([i.strip() for i in contentList[contentList.index(topicList[0]) + 1:contentList.index(topicList[1])]
if i not in  ['', 'RESULTS REVIEW:']])
    elif 'RESULTS REVIEW:' in content:
        review = ' '.join([i.strip() for i in contentList[contentList.index('RESULTS REVIEW:') + 1:contentList.index(topicList[1])]
                      if i not in  ['', 'RESULTS REVIEW:']])  
    assessment = ' '.join([i.strip() for i in contentList[contentList.index(topicList[1]) + 1:contentList.index(topicList[2]) - 1]
                      if i not in ['', ':', 'comments:']])
    details = ' '.join([i.strip() for i in contentList[contentList.index(topicList[2]):-2]
                       if i not in ['',':']])
#     if 'Announcement date:' in contentList[-1] and re.search('Announcement date: (.+?\d{4})', contentList[-1]):
#         announcementDate = re.search('Announcement date: (.+?\d{4})', contentList[-1]).group(1)
#     else:
#         announcementDate = ''
    return review, assessment, details

In [ ]:
def groupText2(self, content):
    topicList = ['RESULTS REVIEW:', 'ASSESSMENT:', 'MORE DETAILS:']

    contentList = content.split('\n')
    review = ' '.join([i.strip() for i in contentList[contentList.index(topicList[0]) + 1:contentList.index(topicList[1])]
                      if i!= ''])
    assessment = ' '.join([i.strip() for i in contentList[contentList.index(topicList[1]) + 1:contentList.index(topicList[2]) - 1]
                      if i not in ['', ':', 'comments:']])
    contentList2 = [i for i in contentList if i != '']
    details = ' '.join([re.sub('Ł', '', i.strip()) for i in contentList2[contentList2.index(topicList[2]) + 
                                      1:] if i not in [':']])
    if 'Announcement date:' in contentList[-1] and re.search('Announcement date: (.+?\d{4})', contentList[-1]):
        announcementDate = re.search('Announcement date: (.+?\d{4})', groupText(pdf2)[-1]).group(1)
    else:
        announcementDate = ''
    return review, assessment, details, announcementDate

In [10]:
cutSentenceFile = "D:/Users/figohjs/Documents/JOPPK/Data/External/CutSentence.xlsx"
df = pd.read_excel(cutSentenceFile)
cutSentenceDict = df.set_index('Original').to_dict()['Replacement']

def cutSentence(text):
    #cut sentence
    if re.search('|'.join(cutSentenceDict.keys()), text):
        matchList = re.findall('|'.join(cutSentenceDict.keys()), text)
        for match in matchList:
            text = re.sub(match, cutSentenceDict[match], text)     
    return text

#between -1 (most extreme negative) and +1 (most extreme positive)
def giveSentimentScore(text):
    analyzer = SentimentIntensityAnalyzer()
    text = cutSentence(text)
    sentimentscore = analyzer.polarity_scores(text)
    score = sentimentscore['compound']
    return score

def magnifyScore(text, overriding = False):
        splitPoint = 'Share price reaction:'
        firstText = text.split(splitPoint)[0]
        # secondText = re.sub('\-','minus ', re.sub('\+', 'plus ', text.split(splitPoint)[1]))
        #if no share price reaction in section of more details
        if len(text.split(splitPoint)) == 1:
            secondText = re.sub('\-','minus ', re.sub('\+', 'plus ', text.split(splitPoint)[0]))
        else:
            secondText = re.sub('\-','minus ', re.sub('\+', 'plus ', text.split(splitPoint)[1]))
        maxNum = 5
        magnifier = 3

        weightFirstText = 1/(magnifier + 1)
        weightSecondText = magnifier/(magnifier + 1)
        #use number in percent as weightage
        if re.search('-*(\d+(\.\d+)*)%', secondText):
            numberPercent = float(re.search('(\d+(\.\d+)*)%', secondText).group(1))
            print("Percent:%s \n"%numberPercent)
            PositivePhraseList = ["rose", "rose by", "soared", "soared as much as"]
            NegativePhraseList = ["declined", "declined by", "slumped", "slumped by", 
                                  "fell", "fell by"]
            #check if there is positive/negative phrase preceding numberPercent
            if re.search("(" + "|".join(PositivePhraseList + NegativePhraseList) + ")\s%s"%numberPercent, secondText):
                secondText = re.search("(" + "|".join(PositivePhraseList + NegativePhraseList) + ")\s%s"%numberPercent, secondText).group(1)
            else:
                keyPhrase = ''
            #if number is more than maxNum
            if numberPercent >= maxNum:
                #if overriding is selected
                if overriding == True:
                    #print('Sentiment (second part text): %s'%self.giveSentimentScore(secondText))
                    return giveSentimentScore(secondText)
                else:
                    numberWeight = 1
                    #if secondtext is still not identified, i.e empty:
                    if keyPhrase == '':
                        secondText = re.search('(.*?\d+(\.\d+)*%(\s\w+){0,2})', secondText).group(1)
#                     print("Before Share Price:%s \n"%str(weightFirstText*giveSentimentScore(firstText)))
#                     print("After Share Price: %s \n"%str(weightSecondText*numberWeight*(giveSentimentScore(secondText)/abs(giveSentimentScore(secondText)))))
#                     print("Second Text: %s \n"%str(secondText))
                    if giveSentimentScore(secondText) == 0:
                        return weightFirstText*giveSentimentScore(firstText)
                    else:
                        return weightFirstText*giveSentimentScore(firstText) + weightSecondText*numberWeight*(giveSentimentScore(secondText)/abs(giveSentimentScore(secondText)))
            else:
                numberWeight = numberPercent/maxNum
                #print('Sentiment (second part text): %s'%self.giveSentimentScore(secondText))
                #print('Number of weight: %s'%numberWeight)
                return weightFirstText*giveSentimentScore(firstText) + weightSecondText*numberWeight*giveSentimentScore(secondText)
        else:
            numberWeight = 1
            #print('Sentiment (second part text): %s'%self.giveSentimentScore(secondText))
            #print('Number of weight: %s'%numberWeight)
            return weightFirstText*giveSentimentScore(firstText) + weightSecondText*numberWeight*giveSentimentScore(secondText)

### Healthcare[CVS UN Q3 2018]

In [ ]:
#missing announcement in text so failed group text func

In [3]:
filename = "D:/Users/figohjs/Documents/JOPPK/Data/Raw/Quarterly Earnings PDF 2018/Healthcare/CVS UN Q3 2018 Earnings Update.pdf"
pdf = convertPdfToTxt(filename)

In [5]:
pdf

"RAHSIA\n\nRESULTS REVIEW:\n\n• Beat: Q3 results beat estimates by 0.99%, posting earnings per share of USD1.73, rising 15.3% YoY (Q3 2017: USD1.50).\nRevenue was at USD47.27 billion, beating estimates by 0.14%, increasing 2.4% driven by growth in pharmacy network and\nmail choice claim volume as well as brand inflation, offsetting continued price compression. The company affirmed its 2018\nguidance to deliver full year Adjusted EPS of USD6.98 to USD7.08.\n\n• Share price reaction: +5.71% as the company reported results beating estimates amid online competition. \n\n.\n\n• Going forward, CVS is best positioned to offset slower growth in drug pricing, while pending deal with Aetna provides\nsynergies from lowering the total cost of care for patients.\n\n• The company continues to face pressure from Amazon's entry into pharmacy business, but the recent performance bodes\nwell for the company ahead of the closing of Aetna deal. \n\n• Analysts’ comments: Revenues were ahead of expectations

In [7]:
reviewText = "• Beat: Q3 results beat estimates by 0.99%, posting earnings per share of USD1.73, rising 15.3% YoY (Q3 2017: USD1.50).\nRevenue was at USD47.27 billion, beating estimates by 0.14%, increasing 2.4% driven by growth in pharmacy network and\nmail choice claim volume as well as brand inflation, offsetting continued price compression. The company affirmed its 2018\nguidance to deliver full year Adjusted EPS of USD6.98 to USD7.08.\n\n• Share price reaction: +5.71% as the company reported results beating estimates amid online competition. \n\n.\n\n"
assessmentText = "• Going forward, CVS is best positioned to offset slower growth in drug pricing, while pending deal with Aetna provides\nsynergies from lowering the total cost of care for patients.\n\n• The company continues to face pressure from Amazon's entry into pharmacy business, but the recent performance bodes\nwell for the company ahead of the closing of Aetna deal. \n\n• Analysts’ comments: Revenues were ahead of expectations and EPS was mainly driven by below the line items (lower\ninterest expense and tax rate). The more specific timeline around the Aetna deal should be viewed as positive as it diminish\nuncertainty surrounding the deal."
detailsText = "• Increase in pharmacy network claim volume was due to an increase in net new business, while the increase in mail choice\nclaim volume was driven by the continued adoption of Maintenance Choice offerings and an increase in specialty pharmacy\nclaims. \n\n• Retail/LTC segment revenue increased 6.4% to USD20.9 billion, due to an increase in same store prescription volume due\nto adoption of CVS's Patient Care Programs, alliances with PBMs and health plans. Same store sales was driven by increase\nin prescription volumes, partially offset by reimbursement pressure and negative impact of generic introduction. \n\n• Operating profit in Q3 declined 5.8% due to increase in acquisition related transaction and integration costs, increase in\noperating expenses due to investment of savings from the Tax Cuts and Jobs Act in wages and benefits, as well as an\nincrease in operating expenses associated with growth in the business. \n\n• CVS continue to expect full year GAAP operating profit to decline 39% to 41%, reflecting goodwill impairment in Q2. Diluted\nEPS is expected to be at USD1.40 to USD1.50 and Adjusted EPS of USD6.98 to USD7.08. \n\n• CVS and Aetna have agreed with the US Department of Justice (DoJ), to divest Aetna's standalone Medicare Part D\nprescription drug plans. Closing of the divestiture is subject to the closing of CVS's proposed acquisition of Aetna. There are\nno remaining antitrust impediment to closing of the proposed acquisition, thus the company expect to close the transaction\nbefore Thanksgiving (22 Nov)."

In [11]:
reviewScore = magnifyScore(reviewText)

Percent:5.71 



In [12]:
assessmentScore = giveSentimentScore(assessmentText)

detailsScore = giveSentimentScore(detailsText)

In [14]:
0.4*assessmentScore + 0.4*reviewScore + 0.2*detailsScore

0.65762